In [54]:
# %load 'graph.py'
import networkx as nx
from sympy.combinatorics.permutations import Permutation
from math import factorial
import random

from cube import RubiksCube
from cube import move_rubiks_cube


class Node(object):

    def __init__(self, rubiks_cube, mov, parent_mov):
        self.rubiks_cube = rubiks_cube
        self.mov = mov
        self.parent_mov = parent_mov


class RCGraph(object):

    def __init__(self):
        self._graph = nx.Graph()
        self.root = Node(RubiksCube(), -1, -1)
        self._graph.add_node(self.root)

    def generate_adjacents(self, node):
        """
        Returns the node adjacent list
        """
        nodes = self._get_possible_adjacents(node)
        while(not self._is_validate_adjacents(nodes)):
            nodes = self._get_possible_adjacents(node)
        for n in nodes:
            self._add_node(n, node)
        return nodes

    def _get_possible_adjacents(self, node):
        """
        Returns a possible list of adjacents.
        """
        # List of 12 or 11 random movements, one for each child
        p_mov = node.parent_mov

        # The permutation of 12 movements (0, 1, 2, ..., 11) to ensure that
        # there is no equal adjacent nodes
        moves = Permutation.unrank_nonlex(12, random.randint(1,
                                          factorial(12))).list()

        if p_mov > 0:
            # Removes the movement which creates a adjacent equals to the
            # node's parent
            moves.remove(p_mov)

        return [Node(move_rubiks_cube(node.rubiks_cube, m), m, node.mov) for m in moves]

    def _is_validate_adjacents(self, nodes):
        """
        Checks if the generated adjacents is valid,
        the list of nodes will be valid if it none of its nodes already exist \
        in the graph.
        """
        for rb in nodes:
            if self._graph.has_node(rb):
                return False
        return True

    def _add_node(self, node, parent=None):
        """
        Adds a new node in the graph.
        """
        self._graph.add_node(node)
        if parent:
            self._graph.add_edge(node, parent)
        else:
            self._graph.add_edge(node, self.root)


In [55]:
graph = RCGraph()

In [56]:
graph.root.rubiks_cube.cube

[[['F1', 'F2', 'F3'], ['F4', 'F5', 'F6'], ['F7', 'F8', 'F9']],
 [['B1', 'B2', 'B3'], ['B4', 'B5', 'B6'], ['B7', 'B8', 'B9']],
 [['D1', 'D2', 'D3'], ['D4', 'D5', 'D6'], ['D7', 'D8', 'D9']],
 [['L1', 'L2', 'L3'], ['L4', 'L5', 'L6'], ['L7', 'L8', 'L9']],
 [['R1', 'R2', 'R3'], ['R4', 'R5', 'R6'], ['R7', 'R8', 'R9']],
 [['U1', 'U2', 'U3'], ['U4', 'U5', 'U6'], ['U7', 'U8', 'U9']]]

In [57]:
adjs = graph.generate_adjacents(graph.root)
print [adj.mov for adj in adjs]
len(adjs)

[3, 0, 7, 5, 10, 8, 1, 9, 4, 2, 6, 11]


12

In [58]:
adjs2 = graph.generate_adjacents(adjs[0])
print adjs[0].mov
print [adj.mov for adj in adjs2]
len(adjs2)

3
[1, 7, 5, 9, 10, 0, 11, 4, 8, 3, 6, 2]


12

In [59]:
adjs3 = graph.generate_adjacents(adjs2[0])
print adjs2[0].mov
print [adj.mov for adj in adjs3]
len(adjs3)

1
[5, 6, 11, 9, 2, 7, 1, 8, 0, 10, 4]


11